In [1]:
from joblib import dump, load
from newspaper import Article, Config
import numpy as np


In [2]:
#dt = load('../../models/dt_pipeline_id1.joblib')
#rf = load('../../models/rf_pipeline_id1.joblib')
#svc = load('../../models/calSVC_pipeline_id2_0_87.joblib')
ens = load('../../models/ensemble_pipeline_id1_0_89.joblib')

In [3]:
def ujArticle(modell, url):
    try:
        # 1. Cikk letöltése és parse-olása
        article = Article(url)
        article.download()
        article.parse()
        
        # Ha üres a szöveg, felesleges folytatni
        if not article.text:
            return "Hiba: A cikk szövege üres vagy nem sikerült letölteni."

        # Cím és szöveg összefűzése (ez a bemenet)
        teljes_szoveg = article.title + " " + article.text
        
        # 2. A DÖNTÉS (Hard Voting esetén ez a mérvadó 0 vagy 1)
        tipp = modell.predict([teljes_szoveg])[0]
        
        # 3. A BIZONYOSSÁG kiszámítása (Trükkös rész)
        valoszinuseg = 0.0
        
        try:
            # A) Megpróbáljuk a hivatalos utat (Soft votingnál működik)
            probs = modell.predict_proba([teljes_szoveg])[0]
            valoszinuseg = probs[tipp] # A nyertes kategória valószínűsége
            
        except AttributeError:
            # B) Hard voting esetén nincs predict_proba -> Kézzel számoljuk
            # Feltételezzük, hogy a 'modell' egy Pipeline
            
            # Kiemeljük a lépéseket a pipeline-ból
            # modell.steps[0][1] -> A Vectorizer (pl. TF-IDF)
            # modell.steps[-1][1] -> A Classifier (maga a VotingClassifier)
            vectorizer = modell.steps[0][1]
            clf = modell.steps[-1][1]
            
            # Átalakítjuk a szöveget számokká
            X_vec = vectorizer.transform([teljes_szoveg])
            
            # Végigmegyünk a belső modelleken (SVC, GradientBoosting)
            belso_probs = []
            if hasattr(clf, 'estimators_'):
                for estimator in clf.estimators_:
                    try:
                        # Minden belső modelltől megkérdezzük a véleményét
                        p = estimator.predict_proba(X_vec)[0]
                        belso_probs.append(p[tipp])
                    except AttributeError:
                        # Pl. LinearSVC-nél nincs predict_proba, itt átugorjuk
                        pass
            
            # Átlagoljuk a belső véleményeket
            if belso_probs:
                valoszinuseg = np.mean(belso_probs)
            else:
                valoszinuseg = 1.0 

        kategoriak = {0: "Független", 1: "Propaganda"}
        eredmeny_szoveg = kategoriak.get(tipp, "Ismeretlen")
        
        # Visszatérünk a listával: [Szöveg, Szám]
        return [eredmeny_szoveg, valoszinuseg]

    except Exception as e:
        print(f"Hiba: {e}")
        return ["Hiba", 0.0]

In [4]:
def ujIras(modell, text):
    try:
        # 1. A DÖNTÉS (Hard Voting esetén ez a mérvadó 0 vagy 1)
        # Itt közvetlenül a bemeneti szöveget adjuk át
        tipp = modell.predict([text])[0]
        
        # 2. A BIZONYOSSÁG kiszámítása
        valoszinuseg = 0.0
        
        try:
            # A) Megpróbáljuk a hivatalos utat
            probs = modell.predict_proba([text])[0]
            valoszinuseg = probs[tipp]
            
        except AttributeError:
            # B) Hard voting esetén kézzel számoljuk ki a belső modellekből
            
            # Kiemeljük a lépéseket a pipeline-ból
            vectorizer = modell.steps[0][1]
            clf = modell.steps[-1][1]
            
            # Átalakítjuk a szöveget számokká
            X_vec = vectorizer.transform([text])
            
            belso_probs = []
            if hasattr(clf, 'estimators_'):
                for estimator in clf.estimators_:
                    try:
                        # Minden belső modelltől megkérdezzük a véleményét
                        p = estimator.predict_proba(X_vec)[0]
                        belso_probs.append(p[tipp])
                    except AttributeError:
                        pass
            
            # Átlagoljuk a belső véleményeket
            if belso_probs:
                valoszinuseg = np.mean(belso_probs)
            else:
                valoszinuseg = 1.0 # Ha nincs adat, 100%-nak vesszük a döntést

        # 3. Eredmény formázása
        kategoriak = {0: "Független", 1: "Propaganda"}
        eredmeny_szoveg = kategoriak.get(tipp, "Ismeretlen")
        
        # FONTOS: Listát adunk vissza: [Címke, Valószínűség(float)]
        return [eredmeny_szoveg, valoszinuseg]

    except Exception as e:
        print(f"Hiba a szöveg feldolgozásakor: {e}")
        return ["Hiba", 0.0]

In [5]:
def szazalek(adat):
    if adat[0] == "Független":
        return 1-adat[1]
    else:
        return adat[1]

In [6]:
def kiir(modell, text):
    if 'http' in text:
        data = ujArticle(modell, text)
    else:
        data = ujIras(modell, text)
    
    try:
        cimke = data[0]
        prop_esely = szazalek(data) # Ez pl. 0.3376
        
        szazalek_kiir = round(prop_esely * 100, 2)
        
        # LOGIKAI JAVÍTÁS:
        # Ha a címke Propaganda, de az esély < 50%, akkor ez egy "Gyanús" cikk, de nem biztos.
        if cimke == "Propaganda" and prop_esely < 0.5:
            return f"Propagandára gyanús (bizonytalan találat), {szazalek_kiir}% esély a propagandára{f' | URL: {text}' if 'http' in text else ''}"
            
        return f'{cimke}, {szazalek_kiir}% esély a propagandára{f" | URL: {text}" if "http" in text else ""}'

    except Exception as e:
        return f"Hiba: {e}"

In [7]:
def teszt(modell):
    teszt = []
    with open('../../data/0urls_test.txt', 'r') as f:
        for line in f:
            teszt.append(['Független', line.strip()])
            
    with open('../../data/1urls_test.txt', 'r') as f:
        for line in f:
            teszt.append(['Propaganda', line.strip()])
    talalat = 0
    try:
        for elvart, url in teszt:
            cimke, bizt = ujArticle(modell, url)
            
            # f-string formázás: .2% beszorozza 100-zal és kiteszi a jelet
            print(f"{'! ' if cimke != elvart else ''}{elvart:10} -> Modell tippje: {cimke:10} | Esély a propagandára: {szazalek([cimke, bizt]):.2%} | URL: {url}")
            
            if elvart == cimke:
                talalat += 1
    except:
            for elvart, url in teszt:
                cimke = ujArticle(modell, url)
                
                # f-string formázás: .2% beszorozza 100-zal és kiteszi a jelet
                print(f"{elvart:10} -> Modell tippje: {cimke:10} | URL: {url}")
                
                if elvart == cimke:
                    talalat += 1
    print(f'Találatok száma: {talalat}')
    print(f'Találatok aránya: {talalat/len(teszt)}')

In [8]:
#teszt(ens)

In [11]:
inp = input(": ")

while inp != "stop":
    print(kiir(ens, inp))
    print("----------------------------")
    inp = input(": ")

Propaganda, 94.1% esély a propagandára | URL: https://mandiner.hu/kulfold/2026/01/eddig-turte-ukrajna-amokfutasat-szijjarto-bekerette-ukrajna-magyarorszagi-nagykovetet-video?utm_source=timeline&utm_medium=web&utm_campaign=news2022-2025
----------------------------
Propaganda, 70.57% esély a propagandára | URL: https://mandiner.hu/kulfold/2026/01/atombombakent-robbant-a-hir-donald-trump-megnevezte-mit-ker-ukrajnatol-a-biztonsagert-cserebe-ha-ezt-nem-teljesiti-zelenszkij-lehuzhatja-a-rolot?utm_source=timeline&utm_medium=web&utm_campaign=news2022-2025
----------------------------
Független, 24.8% esély a propagandára | URL: https://444.hu/2026/01/27/tomegevel-csapnak-be-bangladesieket-oroszorszagi-munkat-igerve-az-aldozatok-a-frontra-kerulnek
----------------------------
Független, 24.8% esély a propagandára | URL: https://444.hu/2026/01/27/tomegevel-csapnak-be-bangladesieket-oroszorszagi-munkat-igerve-az-aldozatok-a-frontra-kerulnek
----------------------------
Független, 29.26% esély a 